# Notizbuch für den 'data' Endpunkt & Spezifikation von 'job=true'
Initial ist 'job=false' gesetzt, daher muss dieser Parameter bei der Abfrage eins zu großen Datensatzes angepasst werden.
Der User erhält über einen Input die Möglichkeit, den Job anzustoßen.
Dessen Status wird über den catalogue/jobs Endpunkt überprüft.
Final wird das gecachte resultfile ausgegeben.

In [6]:
import logging
from pygenesis.http_helper import get_data_from_endpoint

In [7]:
# set logging level to INFO to receive more detailed program response
logging.basicConfig(level=logging.INFO)

endpoint = 'data'
method = 'tablefile'
params = {'name':'42151-0001',
          'area':'all',
          'timeslices':20,
          'job':'false',
          'language':'de',}
response = get_data_from_endpoint(endpoint=endpoint, method=method, params=params)

ReadTimeout: HTTPSConnectionPool(host='www-genesis.destatis.de', port=443): Read timed out. (read timeout=15)

In [3]:
print(response)

<Response [42]>
